### Encontrando vias em comum do aledronato e dos genes diferencialmente expressos na osteoporose

In [1]:
import pandas as pd

david_drug= pd.read_csv("../data/external/DAVIDChartReport_raloxifeno.csv")
david_GSE7158= pd.read_csv("../data/external/DAVIDChartReport_GSE7158.csv")
david_GSE56814= pd.read_csv("../data/external/DAVIDChartReport_GSE56814.csv")
david_GSE56815= pd.read_csv("../data/external/DAVIDChartReport_GSE56815.csv")

comuns_drug_GSE7158 = set(david_drug["Term"]) & set(david_GSE7158["Term"])
comuns_drug_GSE7158 = pd.DataFrame({"Genes em comum drug e GSE7158": list(comuns_drug_GSE7158)})

comuns_drug_GSE56814= set(david_drug["Term"]) & set(david_GSE56814["Term"])
comuns_drug_GSE56814 = pd.DataFrame({"Genes em comum drug e GGSE56814": list(comuns_drug_GSE56814)})

comuns_drug_GSE56815 = set(david_drug["Term"]) & set(david_GSE56815["Term"])
comuns_drug_GSE56815 = pd.DataFrame({"Genes em comum drug e GSE56815": list(comuns_drug_GSE56815)})

In [6]:
### Common Genes - Drug/GSE7158
comuns_drug_GSE7158

,Genes em comum drug e GSE7158
0,Signaling by GPCR
1,Interferon gamma signaling
2,Mitotic G1 phase and G1/S transition
3,Cytokine Signaling in Immune system
4,GPCR downstream signalling
5,G alpha (i) signalling events
6,Immune System
7,G alpha (q) signalling events


In [7]:
### Common Genes - Drug/GSE56814
comuns_drug_GSE56814

,Genes em comum drug e GGSE56814
0,Signaling by GPCR
1,Viral Infection Pathways
2,Disease
3,Generic Transcription Pathway
4,ESR-mediated signaling
5,Cellular responses to stress
6,RNA Polymerase II Transcription
7,Infectious disease
8,Senescence-Associated Secretory Phenotype (SASP)
9,Cellular responses to stimuli


In [8]:
### Common Genes - Drug/GSE56814
comuns_drug_GSE56815

,Genes em comum drug e GSE56815
0,Signaling by GPCR
1,Viral Infection Pathways
2,Disease
3,Generic Transcription Pathway
4,ESR-mediated signaling
5,Cellular responses to stress
6,RNA Polymerase II Transcription
7,Infectious disease
8,Senescence-Associated Secretory Phenotype (SASP)
9,Cellular responses to stimuli


In [12]:
import pandas as pd

df = pd.read_csv("../data/processed/Drugs_gene_interactions.csv")

# Cria tabela de ARESTAS
edges_drug_genes = df[["Drug", "gene"]].copy()
edges_drug_genes.columns = ["source", "target"]
edges_drug_genes["interaction"] = "targets"

edges_drug_genes.to_csv("edges_drug_genes.csv", index=False)

# Cria tabela de NÓS

drugs = df["Drug"].drop_duplicates().to_frame()
drugs["type"] = "Drug"
drugs["UniprotID"] = ""
drugs["protein_name"] = ""

genes = df[["gene", "UniprotID", "proteína"]].drop_duplicates().copy()
genes.columns = ["id", "UniprotID", "protein_name"]
genes["type"] = "Gene"

drugs.columns = ["id", "type", "UniprotID", "protein_name"]
nodes_drug_genes = pd.concat([drugs, genes], ignore_index=True)

nodes_drug_genes.to_csv("nodes_drug_genes.csv", index=False)

In [2]:
import pandas as pd

drug = "raloxifeno" #"alendronato" "zolendronico"
genes = "GSE56815"
edges_output = "edges_Drug_and_pathways_" + drug + ".csv"
nodes_output = "nodes_Drug_and_pathways_" + drug + ".csv"

df_vias = pd.read_csv("../data/processed/Pathways_" + drug + "_" + genes + ".csv" )
df_drugs = pd.read_csv("../data/processed/lista_genes_" + drug + ".csv")

df_vias

def normalize(x):
    if pd.isna(x):
        return None
    return str(x).strip().upper()

df_vias["Via de Sinalização"] = df_vias["Via de Sinalização"].map(normalize)
df_vias["Name"] = df_vias["Name"].map(normalize)
df_drugs["Gene"] = df_drugs["Gene"].map(normalize)
df_drugs["Drug"] = df_drugs["Drug"].map(normalize)


edges_via_gene = df_vias[["Via de Sinalização", "Name"]].copy()
edges_via_gene.columns = ["source", "target"]

edges_gene_drug = df_drugs[["Gene", "Drug"]].copy()
edges_gene_drug.columns = ["source", "target"]

edges = pd.concat([edges_via_gene, edges_gene_drug], ignore_index=True)
edges.dropna(subset=["source", "target"], inplace=True)
edges.drop_duplicates(inplace=True)

edges.to_csv(edges_output, index=False)


In [4]:

vias = set(df_vias["Via de Sinalização"].dropna().unique())
genes_vias = set(df_vias["Name"].dropna().unique())
genes_drugs = set(df_drugs["Gene"].dropna().unique())
drugs = set(df_drugs["Drug"].dropna().unique())

genes = genes_vias.union(genes_drugs)


all_nodes_in_edges = set(edges["source"]).union(edges["target"])

nodes_data = []
for node in all_nodes_in_edges:
    if node in vias:
        ntype = "pathway"
    elif node in genes:
        ntype = "Gene"
    elif node in drugs:
        ntype = "drug"
    else:
        ntype = "unknown"  
    nodes_data.append((node, ntype))

nodes = pd.DataFrame(nodes_data, columns=["id", "type"])


genes_info = df_vias[["Name", "UniprotID", "Entrez_ID"]].drop_duplicates()
genes_info.rename(columns={"Name": "id"}, inplace=True)

nodes = nodes.merge(genes_info, on="id", how="left")

nodes.to_csv(nodes_output, index=False)